# **The Scraping Part**

In [ ]:
!apt install -q chromium-chromedriver

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 40 not upgraded.
Need to get 88.3 MB of archives.
After this operation, 294 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 99.0.4844.84-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 99.0.4844.84-0ubuntu0.18.04.1 [77.7 MB]
^C


In [ ]:
pip install -q  twitter selenium webdriver-manager

In [ ]:
!apt-get -q update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
import tweepy
import pandas as pd
from twitter import *
import sys
sys.path.append(".")
import urllib
import re
import math

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import time         

In [ ]:
def printtweetdata(n, ith_tweet):
        print()
        print(f"Tweet {n}:")
        print(f"Username:{ith_tweet[0]}")
        print(f"Description:{ith_tweet[1]}")
        print(f"Location:{ith_tweet[2]}")
        print(f"Following Count:{ith_tweet[3]}")
        print(f"Follower Count:{ith_tweet[4]}")
        print(f"Total Tweets:{ith_tweet[5]}")
        print(f"Retweet Count:{ith_tweet[6]}")
        print(f"Tweet Text:{ith_tweet[7]}")
        print(f"Embedded Link:{ith_tweet[8]}")
        print(f"Hashtags Used:{ith_tweet[9]}")
        print(f"Scraped Title:{ith_tweet[10]}")
        print(f"Scraped Body:{ith_tweet[11]}")
        print(f"Tweet score:{ith_tweet[12]}")

In [ ]:
def get_tweet_score(followers, following, totaltweets, retweetcount):
    
    # factors denoting popularity of a given tweet
    # consider the following order of importance
        # retweetcount
        # followers to following ratio
        # totaltweets by the user account
    
    # normalized follower to following ratio
    ratio = (0.8 * (followers + 1))/(0.2 * (following + 1))

    # weights distribution
    # 60% retweetcount
    wt_rt_count = 0.6
    # 40% followers-to-following ratio
    wt_ratio = 0.4
    # log(totaltweets) since total tweets can be very large
    wt_total = math.log10(totaltweets)

    # calculate score and return
    return wt_rt_count*(retweetcount) + wt_ratio*ratio + wt_total

In [ ]:
def scrape(tweets):
        sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
        wd.maximize_window()
        wd.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        wd.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
 
        # Creating DataFrame using pandas
        db = pd.DataFrame(columns=['username',
                                   'description',
                                   'location',
                                   'following',
                                   'followers',
                                   'totaltweets',
                                   'retweetcount',
                                   'text',
                                   'url',
                                   'hashtags',
                                   'title',
                                   'body',
                                   'score'])
 
        to_remove = ['status','youtube','bloomberg','msn']


 
        list_tweets = [tweet for tweet in tweets]
   
        # Counter to maintain Tweet Count
        i = 1
 
        # we will iterate over each tweet in the
        # list for extracting information about each tweet
        for tweet in list_tweets:
                username = tweet.user.screen_name
                description = tweet.user.description
                location = tweet.user.location
                following = tweet.user.friends_count
                followers = tweet.user.followers_count
                totaltweets = tweet.user.statuses_count
                retweetcount = tweet.retweet_count
                hashtags = tweet.entities['hashtags']
                score = get_tweet_score(followers, following, totaltweets, retweetcount)
 
                # Retweets can be distinguished by
                # a retweeted_status attribute,
                # in case it is an invalid reference,
                # except block will be executed
                try:
                        text = tweet.retweeted_status.full_text
                except AttributeError:
                        text = tweet.full_text
                        
                hashtext = list()
                for j in range(0, len(hashtags)):
                        hashtext.append(hashtags[j]['text'])

                urls = re.findall("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", str(text))
                for u in urls:
                  try:
                    opener = urllib.request.build_opener()
                    request = urllib.request.Request(u)
                    response = opener.open(request)
                    actual_url = response.geturl()
                    url = actual_url.split('?')[0]
                  except:
                    url = u.split('?')[0]
 
                if url not in db['url'].to_list() and not re.search("|".join(to_remove), url):

                  try:
                    wd.get(url)
                    el = wd.find_element(By.TAG_NAME, "body")
                    title = wd.title
                    body = el.text
                  except NoSuchElementException:
                    pass
                  except WebDriverException:
                    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
                    chrome_options = webdriver.ChromeOptions()
                    chrome_options.add_argument('--headless')
                    chrome_options.add_argument('--no-sandbox')
                    chrome_options.add_argument('--disable-dev-shm-usage')
                    wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
                    wd.maximize_window()
                    wd.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
                    wd.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
                    pass

                  # Here we are appending all the
                  # extracted information in the DataFrame
                  ith_tweet = [username, description,
                             location, following,
                             followers, totaltweets,
                             retweetcount, text, url, hashtext, title, body, score]
                  db.loc[len(db)] = ith_tweet
                
                  # Function call to print tweet data on screen
                  printtweetdata(i, ith_tweet)
                  i = i+1
        #result = pd.concat([db, db1], axis=1, join="inner")
        #result = result.drop('text',1)
        #filename = 'scraped_tweets.csv'
        # we will save our database as a CSV file.
        #db.to_csv(filename)
        wd.quit()
        return db

In [ ]:
if __name__ == '__main__':
 
        # Enter your own credentials obtained
        # from your developer account
        print("Enter your Twitter credentials here")
        consumer_key = "UTH2d8XmB2AUO2MDYPbdEr9Ea"
        consumer_secret = "sJ0yuskOWVB3rYi1gZX8oRt1TloBMw72PpK90GdwYTewAYcUgM"
        access_key = "1268193934100049922-RIAc2R3nAON8sZUeWD306h5REqnXdk"
        access_secret = "8hrd1vuJTipPtVDtEahuu5xOC9tYJtAnRGtDHjS2YWyBe"
  
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_key, access_secret)
        api = tweepy.API(auth)
 
        # Enter Hashtag and initial date
        print("Enter Twitter HashTag to search for")
        words = input()+" filter:links"+" filter:verified"
        # + " -filter:retweets"
        print("Enter Date since The Tweets are required in yyyy-mm--dd")
        date_since = input()
 
        # number of tweets you want to extract in one run
        numtweet = 500

        # We are using .Cursor() to search
        # through twitter for the required tweets.
        # The number of tweets can be
        # restricted using .items(number of tweets)
        tweets = tweepy.Cursor(api.search,
                               words, lang="en",
                               since_id=date_since,
                               result_type = "mixed",
                               tweet_mode='extended').items(numtweet)
        
        tweets_popular = tweepy.Cursor(api.search,
                               words, lang="en",
                               since_id=date_since,
                               result_type = "popular",
                               tweet_mode='extended').items(numtweet)


        tweets_df = scrape(tweets)
        tweets_popular_df = scrape(tweets_popular)
        print('Scraping has completed!')

Enter Twitter HashTag to search for
Enter Date since The Tweets are required in yyyy-mm--dd


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


Streaming output truncated to the last 5000 lines.
PODCASTS
NEWSLETTER
ADVERTISING
CONTACT
TERMS
PRIVACY
© 2022 DEAN BLUNDELL'S SPORTS, NEWS, PODCAST NETWORK. ALL RIGHTS RESERVED.
Tweet score:103.06300872866011

Tweet 75:
Username:politicalwire
Description:Founder and Publisher of Political Wire.
Location:New York
Following Count:1076
Follower Count:69020
Total Tweets:156506
Retweet Count:0
Tweet Text:Twitter Re-Examines Elon Musk’s Bid https://t.co/3HSHs96FGI via @politicalwire
Embedded Link:https://t.co/3HSHs96FGI
Hashtags Used:[]
Scraped Title:Twitter Re-Examines Elon Musk’s Bid
Scraped Body:FRONT PAGE
MEMBERS
RESOURCES
ADVERTISE
NEWSLETTER
CONTACT US
You are here: Home / Technology / Twitter Re-Examines Elon Musk’s Bid
Twitter Re-Examines Elon Musk’s Bid
April 24, 2022 at 3:31 pm EDT By Taegan Goddard 40 Comments
“Twitter is re-examining Elon Musk’s $43 billion takeover offer after the billionaire lined up financing for the bid, in a sign the social-media company could be more rece

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DeprecationWarning: use options instead of chrome_options


Streaming output truncated to the last 5000 lines.
NIKE:
Nike Super Spring Sale: Up to 50% off over 1,000 just-reduced styles
BACK TO TOP
Edition
English
中文 (Chinese)
日本語 (Japanese)
WSJ Membership Benefits
Customer Center
Legal Policies
Google Play
App Store
SIGN IN
© 2022 Dow Jones & Company Inc.
All Rights Reserved.
Tweet score:39.19885151902206

Tweet 120:
Username:WSJmarkets
Description:Breaking news, smart analysis and in-depth features on global markets and finance from The Wall Street Journal.
Location:New York, NY
Following Count:487
Follower Count:765553
Total Tweets:85713
Retweet Count:5
Tweet Text:Twitter is expected to weigh in on Elon Musk’s bid when it reports first-quarter earnings Thursday, if not sooner https://t.co/tu48MPM6UF
Embedded Link:https://t.co/tu48MPM6UF
Hashtags Used:[]
Scraped Title:Twitter Re-Examines Elon Musk’s Bid, May Be More Receptive to a Deal - WSJ
Scraped Body:DJIA33811.40
points with a
2.82%▼ S&P 5004271.78
points with a
2.77%▼ U.S. 10 Yr0/32 Yiel

...


Tweet 9:
Username:NYMag
Description:The ideas, people, and cultural currents that are forever reshaping the world.
Subscribe: https://t.co/eO5NQ0hSUF
Get One Great Story: https://t.co/FtV7tlY7aX
Location:New York, NY
Following Count:993
Follower Count:1810253
Total Tweets:209759
Retweet Count:215
Tweet Text:Elon Musk's Twitter bid marks a new phase of his career: the end of Musk as an innovator and the beginning of his Rupert Murdoch era, where he's interested in the transformative power of media to exercise greater influence over society. @KevinTDugan writes https://t.co/l7QTRYCmt0
Embedded Link:https://nymag.com/intelligencer/2022/04/elon-musks-twitter-bid-is-his-rupert-murdoch-move.html
Hashtags Used:[]
Scraped Title:Twitter’s Board Is Said to Seriously Consider Elon Musk’s Bid - The New York Times
Scraped Body:SKIP TO CONTENT
SKIP TO SITE INDEX
Elon Musk Offers
to Buy Twitter
An Unsolicited Bid
How Musk Plans to Pay
Apollo’s Possible Involvement
Twitter’s Counterattack
ADVERTISEME

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DeprecationWarning: use options instead of chrome_options



Tweet 18:
Username:theblaze
Description:News and entertainment for people who love America. Retweets =/= endorsements.
Location:Dallas, TX
Following Count:710
Follower Count:717813
Total Tweets:150054
Retweet Count:590
Tweet Text:Joe Rogan rallies for Elon Musk's possible Twitter purchase: 'He’s the super-intelligent leader-type character that seems to have great ethics and morals' https://t.co/HL6uIskI6r
Embedded Link:https://www.theblaze.com/news/joe-rogan-rallies-for-elon-musks-possible-twitter-purchase
Hashtags Used:[]
Scraped Title:OPINION: Elon Musk should not become the owner of Twitter
Scraped Body:NEWS
OPINION: Elon Musk should not become the owner of Twitter
APRIL 17, 2022
DAVID WEISSMAN
Opinion columnist and Army veteran. Former MAGA, now a Biden…
READ NEXT
SAD INHERITANCE: Donald Trump Jr. outdoes his father in posting incredibly offensive Easter messages
Advertisement Above
Elon Musk recently became the largest shareholder of Twitter — briefly, at least — and since he los

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DeprecationWarning: use options instead of chrome_options



Tweet 19:
Username:OccupyDemocrats
Description:Grassroots political organization & news website supporing progressive Democrats. NY Times reported that our reach dominated Trump on Facebook before his ban.
Location:United States
Following Count:18763
Follower Count:333548
Total Tweets:90702
Retweet Count:475
Tweet Text:After Elon Musk's brother made a $108 million windfall from trading Twitter stock, aggrieved investors thought that they smelled something fishy.
https://t.co/FkTLop8PnC
Embedded Link:https://washingtonpress.com/2022/04/17/lawsuit-angry-investors-file-class-action-suit-against-elon-musk-for-his-twitter-takeover-attempt/
Hashtags Used:[]
Scraped Title:OPINION: Elon Musk should not become the owner of Twitter
Scraped Body:NEWS
OPINION: Elon Musk should not become the owner of Twitter
APRIL 17, 2022
DAVID WEISSMAN
Opinion columnist and Army veteran. Former MAGA, now a Biden…
READ NEXT
SAD INHERITANCE: Donald Trump Jr. outdoes his father in posting incredibly offensive East

In [ ]:
tweets_df

,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score
0,kateconger,"tech reporter for @nytimes, covering cybersecu...","Washington, D.C.",2017,34430,136,0,It's getting serious... https://t.co/FQ2QfnVvK9,https://t.co/FQ2QfnVvK9,[],Twitter’s Board Is Said to Seriously Consider ...,SKIP TO CONTENT\nSKIP TO SITE INDEX\nElon Musk...,29.432647
1,RedState,Unapologetic reporting and opinion by true con...,"Washington, D.C.",356,264330,144380,9,BREAKING: WSJ Reports Twitter Will Reconsider ...,https://t.co/327PD2Cxwk,[],BREAKING: WSJ Reports Twitter Will Reconsider ...,BREAKING: WSJ Reports Twitter Will Reconsider ...,1195.236258
2,indypremium,Support free-thinking journalism and get the b...,,1,4093,56743,0,Musk: serious about making Twitter equivalent ...,https://www.independent.co.uk/independentpremi...,[],Musk serious about making Twitter equivalent o...,Contribute\nSubscribe\nLOGIN\nIndependent Prem...,3279.953912
3,chrismattmann,🚀 Chief Technology and Innovation Officer @NAS...,chrismattmann.eth,2471,19942,72705,0,Twitter is re-examining Elon Musk’s $43 billio...,https://t.co/MxQJ0NNDo4,[],"Twitter Re-Examines Elon Musk’s Bid, May Be Mo...",DJIA33811.40\npoints with a\n2.82%▼ S&P 500427...,17.769655
4,Zigmanfreud,"Podcast Host, Former @Mediaite Senior Columnis...","Los Angeles, CA",6314,56394,220677,5,Twitter said to be 're-examining' Elon Musk's ...,https://www.theblaze.com/news/twitter-reexamin...,[],Twitter re-examining Elon Musk's takeover offe...,Chewing the Fat\nConservative Review with Dani...,22.632276
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335687,101682,912,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3536.100577
201,shapshak,I write and speak about how innovation is bett...,Joburg,670,37912,62020,1,Forget Twitter: the world’s richest person sho...,https://www.businesslive.co.za/fm/opinion/patt...,[],TOBY SHAPSHAK: Elon Musk has more important th...,Business Live\nBusiness Live\nToby Shapshak Co...,95.796109
202,DeccanHerald,Bengaluru & Karnataka's leading news daily for...,India,152,91346,338420,0,#BhavishAggarwal will not invite #ElonMusk to ...,https://www.deccanherald.com/business/why-shou...,"[BhavishAggarwal, ElonMusk]",TOBY SHAPSHAK: Elon Musk has more important th...,Business Live\nBusiness Live\nToby Shapshak Co...,960.792201
203,NationalWorld,"News, explainers, features and comment from jo...",UK,312,2363,18967,0,How did Elon Musk make his billions? Here's ev...,https://t.co/0Ala0ZQL1d,[],How Elon Musk became the richest person in the...,News\nPeople\nWhere does Elon Musk get his mon...,16.362344


In [ ]:
tweets_popular_df

,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score
0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454
1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,[Dogecoin],This Dogecoin transaction happened without acc...,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094
2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466
3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619
4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,Skip to main content\nSkip to navigation\nSupp...,4014.861106
5,DineshDSouza,"I am an author, filmmaker, and host of the Din...",United States,5723,2002134,87482,390,BREAKING: Twitter reconsiders Elon Musk's take...,https://t.co/t9cXLMdNXT,[],BREAKING: Twitter reconsiders Elon Musk’s take...,LOGIN\nSUBSCRIBE NOW\nUse code TPM for up to 6...,798.588320
6,CoinMarketCap,Showcasing the crypto revolution to +340M mont...,Decentralized,1848,5255835,17820,203,Crypto billionaire Sam Bankman-Fried says he w...,https://coinmarketcap.com/alexandria/article/c...,[],Crypto Billionaire Would Be 'Excited' to Help ...,Crypto News\nCrypto Billionaire Would Be 'Exci...,4674.097203
7,nytimes,News tips? Share them here: https://t.co/ghL9O...,New York City,878,52724320,469799,196,Breaking News: Twitter’s board is seriously co...,https://t.co/knjwxAOzSV,[],Twitter’s Board Is Said to Seriously Consider ...,SKIP TO CONTENT\nSKIP TO SITE INDEX\nElon Musk...,96094.732208
8,NYMag,"The ideas, people, and cultural currents that ...","New York, NY",993,1810253,209759,215,Elon Musk's Twitter bid marks a new phase of h...,https://nymag.com/intelligencer/2022/04/elon-m...,[],Twitter’s Board Is Said to Seriously Consider ...,SKIP TO CONTENT\nSKIP TO SITE INDEX\nElon Musk...,3048.211459
9,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203406,464029,138,Elon Musk registers three 'X Holdings' compani...,https://www.foxbusiness.com/markets/elon-musk-...,[],Elon Musk registers three 'X Holdings' compani...,Fox Business\nPersonal Finance\nEconomy\nMarke...,128593.963515


In [ ]:
tweets_df = tweets_df.sort_values(by = "score",ascending = False).reset_index(drop =True)
df_final = pd.concat([tweets_popular_df,tweets_df]).drop_duplicates(subset="url").reset_index(drop =True)
df_final.head()

,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score
0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454
1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,[Dogecoin],This Dogecoin transaction happened without acc...,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094
2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466
3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619
4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,Skip to main content\nSkip to navigation\nSupp...,4014.861106


In [ ]:
filename = 'Twitter_data.csv'
df_final.to_csv(filename)

# Data preprocessing & Tokenization

In [1]:
!pip install -q contractions rank_bm25

     |████████████████████████████████| 287 kB 5.2 MB/s 
     |████████████████████████████████| 106 kB 57.7 MB/s 


In [2]:
# pre-preprocessing for tokenization
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
import contractions
import re
from nltk.corpus import stopwords
from rank_bm25 import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
#Remove URLs from a sample string

def remove_URL(sample):
    return re.sub(r"http\S+", "", sample)

# Replace contractions in string of text
def replace_contractions(text):
    return contractions.fix(text)

# Convert all characters to lowercase from list of tokenized words
def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

#Remove punctuation from list of tokenized words
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

# Remove stop words from list of tokenized words
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words


In [4]:
import pandas as pd
df_final = pd.read_csv('Twitter_data.csv')

In [5]:
df_final.rename(columns = {'Unnamed: 0':'Ranking'}, inplace = True)
df_final

,Ranking,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score
0,0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454
1,1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,['Dogecoin'],This Dogecoin transaction happened without acc...,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094
2,2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466
3,3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619
4,4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,Skip to main content\nSkip to navigation\nSupp...,4014.861106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,222,Jacksonngari_,Communication and Media Student |\nA Gen-Z on ...,"Nyeri, Kenya",1377,2621,4187,0,"“Moving On”, Elon Musk’s cryptic tweet sends p...",https://t.co/jRpmMAGzo6,[],Elon Musk sends out a cryptic tweet amidst his...,"Monday, April 25, 2022\nPOLITICS\nOPINIONS\nFA...",6.666315
223,223,mrtgr,"Chicagoan, ex–S.F., Prague. No. 6 on @MarketWa...",Chicago.,8570,5204,113111,0,Twitter taking a second look at Elon Musk's of...,https://www.marketwatch.com/story/twitter-taki...,[],Twitter taking a second look at Elon Musk's ta...,SKIP TO MAIN CONTENT\nMarketWatch Site Logo A ...,6.025153
224,224,PatricioEspnoza,Emmy Investigative Journo & Traveler of Life! ...,"Phoenix, AZ",818,803,13430,0,Top story: Twitter Re-Examines Elon Musk’s Bid...,https://t.co/vUKNwgM3WD,[],The Tweeted Times - personal newspaper generat...,Loading...\nFeatured\n+ Create newspaper\nPatr...,5.698772
225,225,nicholasadeleon,Sr Reporter @ConsumerReports mostly covering l...,New York City,3209,3456,5060,0,Twitter is basically flat since IPO so maybe h...,https://t.co/gYR7miLOca,[],"Twitter Re-Examines Elon Musk’s Bid, May Be Mo...",DJIA33811.40\npoints with a\n2.82%▼ S&P 500427...,5.427266


## Okapi BM25

In [6]:
token_list = []
for text in df_final["text"]:
  x = remove_URL(text)
  x = replace_contractions(x)
  tokens = to_lowercase(remove_stopwords(remove_punctuation(nltk.word_tokenize(x))))
  token_list = token_list + tokens

# unique tokens:
token_list = list(set(token_list))

In [7]:
# tokenize the bodys of all links

i=0
link_content_tokenized = []
for text in df_final["body"]:
  x = remove_URL(text)
  x = replace_contractions(x)
  tokens = to_lowercase(remove_stopwords(remove_punctuation(nltk.word_tokenize(x))))
  link_content_tokenized.append(tokens)

In [8]:
# matching using bm25
bm25 = BM25Okapi(link_content_tokenized)
doc_scores = bm25.get_scores('elon')
df_final["bm25_score"] = doc_scores
df_final.head()

,Ranking,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score,bm25_score
0,0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454,0.000000
1,1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,['Dogecoin'],This Dogecoin transaction happened without acc...,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094,0.000000
2,2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466,11.720660
3,3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619,0.000000
4,4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,Skip to main content\nSkip to navigation\nSupp...,4014.861106,1.265734


In [9]:
# Ten largest values in BM25
df_final.nlargest(10, ['bm25_score'])


,Ranking,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score,bm25_score
2,2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466,11.720660
159,159,mark_dow,Fiat enthusiast. 🇺🇸 🇮🇹 /s 🇪🇸 🇫🇷 🇵🇹,Behavioral Macro,3890,95279,107125,2,@sspencer_smb Free markets. Freedom of speech....,https://markets.businessinsider.com/news/stock...,[],"Tesla, SpaceX CEO Elon Musk Says 'Short Sellin...",Menu icon A vertical stack of three evenly spa...,45.409536,10.397541
148,148,Info_Activism,We are an international NGO that engages with ...,NaN,1127,48677,67252,2,Jeff Bezos is worth $160bn – yet Congress migh...,https://www.theguardian.com/commentisfree/2022...,[],Jeff Bezos is worth $160bn – yet Congress migh...,Skip to main content\nSkip to navigation\nSupp...,75.074514,6.959871
79,79,latimes,Covering the world with a West Coast perspecti...,"El Segundo, CA",7067,3920276,428459,3,“Maybe a tour of football stadiums. Who knows?...,https://www.latimes.com/entertainment-arts/sto...,[],"For 'Fluffy' Iglesias, Netflix Is a Joke is a ...",Sections\nLOG IN\nSubscribe Now\n$1 for 6 mont...,894.874354,6.394375
88,88,spectator,"Politics, culture and more; weekly since 1828,...",London,707,235168,111260,1,"'Keep your wandering focus firmly on Tesla, Mr...",https://www.spectator.co.uk/article/why-elon-m...,[],Why Elon Musk should forget Twitter and stick ...,Now only £1 a week for your first year\nPlus g...,537.101706,5.143922
71,71,standardnews,Breaking news and updates on top stories from ...,London,856,660906,418698,2,City Comment: Netflix's 35% collapse after its...,https://www.standard.co.uk/business/netflix-te...,[],City comment: Changing of the guard as Netflix...,BREAKING\n7h\nFamily’s fears for missing 14-ye...,1240.720617,4.955596
99,99,MGleiser,"Physicist, essayist, curator of the unknown, a...","Hanover, NH",139,27394,3735,3,Elon Musk made a $43B bid to buy Twitter. He s...,https://sign.moveon.org/petitions/stop-elon-mu...,[],Stop Elon Musk’s Hostile Twitter Takeover! | M...,Skip to main content\nDONATE\nCAMPAIGN TIPS\nS...,318.458005,4.534685
142,142,robertliefeld,"Best selling Author/Creator of DEADPOOL, CABLE...",NaN,1874,112463,155554,8,JUST IN: Twitter Taking ‘Fresh Look’ at Elon M...,https://t.co/GThjurHxJI,[],Twitter Taking 'Fresh Look' at Elon Musk Offer...,"Mediaite .mediaite-shadow {fill: rgba(0,0,0,.2...",105.961161,4.222294
150,150,CityNewsHFX,"@NEWS957 is now @CityNewsHFX, follow us here f...","Halifax, Nova Scotia, Canada",1557,65658,135084,0,There is a very real risk of losing ideas in t...,https://halifax.citynews.ca/local-news/a-twitt...,[],"A Twitter takeover could restrict speech, not ...",Skip to content\nListen: CITY News Everywhere\...,72.559615,3.335627
193,193,JimPethokoukis,Substack: https://t.co/uCRWhJEUKZ @AEI Fellow....,"Washington, DC",7151,56665,97386,3,🚀 Elon Musk could become the world's first tri...,https://t.co/c6ahsEHrco,[],🚀 Elon Musk could become the world's first tri...,"Faster, Please!\nSubscribe\nSign in\n🚀 Elon Mu...",19.465454,2.894304


## Tf-idf

In [10]:
pip install -q num2words

     |████████████████████████████████| 101 kB 4.2 MB/s 


In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [12]:
def convert_numbers(data):
    processed_data = []
    for doc in data:
      l_token = []
      for w in doc:
          if type(w) == int:
            try:
                w = num2words(int(w))
                l_token.append(w)
            except:
                a = 0
          else:
            l_token.append(w)
      processed_data.append(l_token)
      #new_text = new_text + " " + w
      #new_text = np.char.replace(new_text, "-", " ")
    return processed_data

In [13]:
link_content_tokenized = convert_numbers(link_content_tokenized)

In [14]:
token_list = convert_numbers(token_list)

In [15]:
len(link_content_tokenized[0])

674

In [16]:
#Calculating Document Frequency
DF = {}
for i in range(len(df_final)):
    tokens = link_content_tokenized[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = token_list[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [17]:
#TF-idf for the body
doc = 0

tf_idf = {}

for i in range(len(df_final)):
    
    tokens = link_content_tokenized[i]
    
    counter = Counter(tokens + token_list[i])
    words_count = len(tokens + token_list[i])
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((len(df_final)+1)/(df+1))
      
        tf_idf[doc, token] = tf*idf

    doc += 1

In [18]:
def get_key(val):
    for key, value in tf_idf.items():
         if key[1] == val:
             return value
 
    return "key doesn't exist"

print(get_key('elon'))

0.000581744416635072


In [19]:
alpha = 0.3
for i in tf_idf:
    tf_idf[i] *= alpha

In [20]:
query = input("Enter the search query")
query_list = []
for text in query:
  x = remove_URL(text)
  x = replace_contractions(x)
  tokens_ = to_lowercase(remove_stopwords(remove_punctuation(nltk.word_tokenize(x))))
  query_list = query_list + tokens_

# unique tokens:
query_list = list(set(query_list))

query_weights = {}

for key in tf_idf:
  if key[1] in tokens:
    try:
      query_weights[key[0]] += tf_idf[key]
    except:
      query_weights[key[0]] = tf_idf[key]


Enter the search queryelon


In [21]:
tf_idf_list = query_weights.values()
tf_idf_list = list(tf_idf_list)
df_final["tf_idf_score"] = tf_idf_list

In [22]:
df_final.head()

,Ranking,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score,bm25_score,tf_idf_score
0,0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454,0.000000,0.057132
1,1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,['Dogecoin'],This Dogecoin transaction happened without acc...,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094,0.000000,0.023402
2,2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466,11.720660,0.036359
3,3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619,0.000000,0.057428
4,4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,Skip to main content\nSkip to navigation\nSupp...,4014.861106,1.265734,0.053465


In [27]:
# Ten largest values in BM25
df_final[['title', 'url','tf_idf_score']].nlargest(10, ['tf_idf_score'])

,title,url,tf_idf_score
226,Twitter Now More Open To Negotiating With Elon...,https://www.forbes.com/sites/masonbissada/2022...,0.517754
175,Twitter Reconsiders Elon Musk's Bid After Conf...,https://www.businessinsider.com/twitter-recons...,0.103675
212,Twitter Said to Be Re-examining Elon Musk's $4...,https://www.cnet.com/tech/twitter-said-to-be-r...,0.100818
90,Twitter under shareholder pressure to seek dea...,https://www.theglobeandmail.com/business/inter...,0.093064
128,Exclusive: Twitter under shareholder pressure ...,https://www.reuters.com/business/exclusive-twi...,0.089581
64,Twitter May End Up Taking Up Elon Musk’s Offer...,https://www.thedailybeast.com/twitter-may-end-...,0.085425
182,Twitter is reportedly reconsidering Elon Musk'...,https://fortune.com/2022/04/24/twitter-reconsi...,0.084111
161,BREAKING: Twitter reconsiders Elon Musk’s take...,https://t.co/b7SKdvU6xY,0.082108
93,BREAKING: Twitter reconsiders Elon Musk’s take...,https://t.co/CJmUgro8I0,0.080805
105,BREAKING: Twitter reconsiders Elon Musk’s take...,https://t.co/uxRkMG0VmI,0.080805


## Tf-idf with cosine similarity

In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
D = np.zeros((len(df_final), len(DF)))
total_vocab = [x for x in DF]
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [ ]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((len(df_final)+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
query = input("Enter the search query")
query_list = []
for text in query:
  x = remove_URL(text)
  x = replace_contractions(x)
  tokens_ = to_lowercase(remove_stopwords(remove_punctuation(nltk.word_tokenize(x))))
  query_list = query_list + tokens_

# unique tokens:
query_list = list(set(query_list))

d_cosines = []
query_vector = gen_vector(query_list)
    
for d in D:
  d_cosines.append(cosine_sim(query_vector, d))

df_final["cos_sim_score"] = d_cosines

Enter the search queryelon


## PyTerrier

In [ ]:
pip install -q python-terrier

     |████████████████████████████████| 97 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
     |████████████████████████████████| 69 kB 5.8 MB/s 
     |████████████████████████████████| 297 kB 48.3 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
     |████████████████████████████████| 294 kB 55.4 MB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.4 MB 49.6 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 126 kB 59.2 MB/s 
     |████████████████████████████████| 287 kB 59.9 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 


In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
df_final.insert(12, 'docno', range(1, 1 + len(df_final)))

In [ ]:
df_final['docno'] = df_final['docno'].astype(str)

In [ ]:
indexer = pt.DFIndexer("./index_docs", overwrite=True)
index_ref = indexer.index(df_final["body"], df_final["docno"])
index_ref.toString()

'./index_docs/data.properties'

In [ ]:
index = pt.IndexFactory.of(index_ref)

# check index type
type(index)

jnius.reflect.org.terrier.structures.Index

In [ ]:
str_tokens = "elon"

#### PL2

In [ ]:
br = pt.BatchRetrieve(index, wmodel="PL2")
pl2 = br.search(str_tokens)
pl2

,qid,docid,docno,rank,score,query
0,1,79,80,0,4.121969,elon
1,1,171,172,1,2.679921,elon
2,1,62,63,2,2.580473,elon
3,1,77,78,3,2.170610,elon
4,1,71,72,4,2.155695,elon
...,...,...,...,...,...,...
211,1,85,86,211,0.341665,elon
212,1,212,213,212,0.341544,elon
213,1,216,217,213,0.341515,elon
214,1,142,143,214,0.341505,elon


#### InL2

In [ ]:
br = pt.BatchRetrieve(index, wmodel="InL2")
inl2 = br.search(str_tokens)
inl2

,qid,docid,docno,rank,score,query
0,1,15,16,0,0.072001,elon
1,1,121,122,1,0.071701,elon
2,1,41,42,2,0.070686,elon
3,1,197,198,3,0.070486,elon
4,1,101,102,4,0.070082,elon
...,...,...,...,...,...,...
211,1,71,72,211,0.045359,elon
212,1,77,78,212,0.045242,elon
213,1,62,63,213,0.042103,elon
214,1,171,172,214,0.041359,elon


## Kendal Tau Metric

In [ ]:
s = list(range(1,len(df_final)+1))
df_final["doc_no"] = s
df_final.head()

,Ranking,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,docno,body,score,bm25_score,tf_idf_score,cos_sim_score,doc_no
0,0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,1,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454,0.000000,0.057458,0.00000,1
1,1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,['Dogecoin'],This Dogecoin transaction happened without acc...,2,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094,0.000000,0.023257,0.00000,2
2,2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",3,Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466,11.720660,0.036359,0.07685,3
3,3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,4,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619,0.000000,0.057405,0.00000,4
4,4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,5,Skip to main content\nSkip to navigation\nSupp...,4014.861106,1.265734,0.053465,0.00111,5


In [ ]:
df_final.drop("Ranking", axis =1, inplace=True)
df_final

,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,docno,body,score,bm25_score,tf_idf_score,cos_sim_score,doc_no
0,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,1,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454,0.000000,0.057458,0.000000,1
1,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,['Dogecoin'],This Dogecoin transaction happened without acc...,2,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094,0.000000,0.023257,0.000000,2
2,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",3,Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466,11.720660,0.036359,0.076850,3
3,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,4,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619,0.000000,0.057405,0.000000,4
4,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,5,Skip to main content\nSkip to navigation\nSupp...,4014.861106,1.265734,0.053465,0.001110,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Jacksonngari_,Communication and Media Student |\nA Gen-Z on ...,"Nyeri, Kenya",1377,2621,4187,0,"“Moving On”, Elon Musk’s cryptic tweet sends p...",https://t.co/jRpmMAGzo6,[],Elon Musk sends out a cryptic tweet amidst his...,223,"Monday, April 25, 2022\nPOLITICS\nOPINIONS\nFA...",6.666315,0.000000,0.050559,0.000000,223
223,mrtgr,"Chicagoan, ex–S.F., Prague. No. 6 on @MarketWa...",Chicago.,8570,5204,113111,0,Twitter taking a second look at Elon Musk's of...,https://www.marketwatch.com/story/twitter-taki...,[],Twitter taking a second look at Elon Musk's ta...,224,SKIP TO MAIN CONTENT\nMarketWatch Site Logo A ...,6.025153,0.000000,0.056364,0.000000,224
224,PatricioEspnoza,Emmy Investigative Journo & Traveler of Life! ...,"Phoenix, AZ",818,803,13430,0,Top story: Twitter Re-Examines Elon Musk’s Bid...,https://t.co/vUKNwgM3WD,[],The Tweeted Times - personal newspaper generat...,225,Loading...\nFeatured\n+ Create newspaper\nPatr...,5.698772,0.000000,0.061835,0.000000,225
225,nicholasadeleon,Sr Reporter @ConsumerReports mostly covering l...,New York City,3209,3456,5060,0,Twitter is basically flat since IPO so maybe h...,https://t.co/gYR7miLOca,[],"Twitter Re-Examines Elon Musk’s Bid, May Be Mo...",226,DJIA33811.40\npoints with a\n2.82%▼ S&P 500427...,5.427266,1.163018,0.063480,0.002075,226


### Calculating Kendall Rank correlation (base rank, bm25 rank)

In [ ]:
base_rank = list(df_final["doc_no"].astype(int))
print(base_rank)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [ ]:
BM25_final = df_final.sort_values("bm25_score", ascending = False)
new_rank = list(BM25_final["doc_no"].astype(int))
print(new_rank)

[3, 160, 149, 80, 89, 72, 100, 143, 151, 194, 107, 63, 55, 202, 21, 5, 204, 36, 70, 191, 76, 128, 211, 105, 195, 124, 188, 62, 81, 103, 201, 187, 141, 215, 93, 164, 189, 197, 205, 226, 168, 150, 221, 175, 60, 167, 220, 222, 92, 154, 130, 190, 163, 88, 155, 212, 210, 206, 97, 178, 95, 98, 129, 153, 137, 133, 152, 145, 139, 140, 142, 136, 144, 135, 157, 131, 146, 147, 148, 134, 138, 132, 156, 1, 158, 209, 193, 196, 198, 199, 200, 203, 207, 208, 213, 159, 214, 216, 217, 218, 219, 223, 224, 225, 192, 186, 185, 184, 161, 162, 165, 166, 169, 170, 171, 172, 127, 174, 176, 177, 179, 180, 181, 182, 183, 173, 114, 126, 42, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 43, 125, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 30, 29, 28, 27, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 54, 56, 57, 94, 99, 101, 102, 104, 106, 108, 109, 110, 111, 112, 113, 2, 115, 116, 117, 118, 119, 120, 121, 122, 123, 96, 91, 58, 90, 59, 61, 64, 65, 66, 67, 68, 69, 71, 73, 74, 75, 77, 78, 79, 8

In [ ]:
from scipy.stats import kendalltau

In [ ]:
corr, _ = kendalltau(base_rank, new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: -0.24268


### Calculating Kendall Rank correlation (base rank, TF-IDF rank)

In [ ]:
TF_IDF_final = df_final.sort_values("tf_idf_score", ascending = False)
new_rank = list(TF_IDF_final["doc_no"].astype(int))
print(new_rank)

[227, 176, 213, 91, 129, 183, 65, 94, 161, 121, 207, 120, 104, 144, 106, 67, 162, 6, 46, 179, 8, 169, 9, 64, 143, 24, 10, 82, 203, 28, 185, 26, 51, 125, 113, 200, 20, 40, 14, 172, 33, 178, 197, 226, 164, 168, 205, 71, 93, 189, 165, 127, 211, 76, 105, 124, 191, 166, 208, 195, 225, 188, 62, 142, 201, 141, 81, 103, 222, 210, 175, 154, 221, 63, 167, 187, 220, 88, 206, 150, 97, 155, 163, 215, 92, 75, 212, 130, 123, 60, 190, 21, 174, 78, 74, 153, 116, 34, 193, 43, 4, 1, 177, 199, 25, 224, 53, 29, 146, 11, 56, 87, 12, 5, 102, 68, 37, 59, 223, 45, 216, 58, 96, 48, 139, 128, 159, 192, 196, 152, 117, 209, 27, 15, 70, 72, 180, 170, 112, 119, 156, 66, 80, 158, 214, 204, 36, 135, 126, 101, 218, 30, 31, 49, 32, 41, 184, 194, 84, 83, 39, 145, 198, 115, 86, 38, 57, 69, 22, 61, 100, 217, 160, 219, 55, 110, 89, 85, 77, 118, 133, 13, 114, 147, 54, 79, 7, 131, 173, 47, 108, 202, 3, 148, 23, 42, 90, 16, 35, 186, 157, 132, 44, 98, 149, 73, 111, 182, 138, 18, 151, 17, 19, 181, 50, 122, 99, 52, 137, 95, 134, 

In [ ]:
corr, _ = kendalltau(base_rank, new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: -0.10421


### Calculating Kendall Rank correlation (base rank, get_tweet_score function rank)

In [ ]:
scoring_fn_final = df_final.sort_values("score", ascending = False)
new_rank = list(scoring_fn_final["doc_no"].astype(int))
print(len(new_rank))

227


In [ ]:
corr, _ = kendalltau(base_rank, new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: 0.92484


In [ ]:
corr, _ = kendalltau(base_rank, new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: 0.92484


### Calculating Kendall Rank correlation (base rank, PL2 rank)

In [ ]:
new_rank = list(pl2["docno"].astype(int))
print(len(new_rank))

216


In [ ]:
# Calculating Kendall Rank correlation
corr, _ = kendalltau(base_rank[:216], new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: 0.11344


### Calculating Kendall Rank correlation (base rank, inl2 rank)

In [ ]:
new_rank = list(inl2["docno"].astype(int))
print(len(new_rank))

216


In [ ]:
# Calculating Kendall Rank correlation
corr, _ = kendalltau(base_rank[:216], new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: -0.01361


### Calculating Kendall Rank correlation (base rank, TF-IDF with cosine similarity rank)

In [ ]:
TF_IDF_cos = df_final.sort_values("cos_sim_score", ascending = False)
new_rank = list(TF_IDF_cos["doc_no"].astype(int))
print(new_rank)

[3, 80, 160, 151, 130, 63, 107, 210, 163, 60, 164, 93, 187, 72, 149, 81, 103, 62, 211, 191, 76, 141, 105, 92, 220, 212, 221, 195, 190, 88, 167, 206, 168, 150, 55, 226, 189, 205, 215, 178, 21, 95, 202, 5, 188, 124, 155, 175, 222, 201, 154, 197, 97, 204, 70, 128, 36, 98, 132, 136, 133, 131, 156, 157, 158, 134, 135, 129, 137, 153, 152, 145, 139, 140, 142, 143, 144, 148, 147, 146, 159, 138, 1, 161, 209, 194, 196, 198, 199, 200, 203, 207, 208, 213, 162, 214, 216, 217, 218, 219, 223, 224, 225, 193, 192, 186, 185, 165, 166, 169, 170, 171, 172, 173, 126, 174, 176, 177, 179, 180, 181, 182, 183, 184, 127, 114, 125, 42, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 43, 56, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 30, 29, 28, 27, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 54, 57, 123, 111, 96, 99, 100, 101, 102, 104, 106, 108, 109, 110, 112, 58, 113, 2, 115, 116, 117, 118, 119, 120, 121, 122, 94, 91, 90, 89, 59, 61, 64, 65, 66, 67, 68, 69, 71, 73, 74, 75, 77, 78, 79, 8

In [ ]:
corr, _ = kendalltau(base_rank, new_rank)
print('Kendall Rank correlation: %.5f' % corr)

Kendall Rank correlation: -0.25687


## RBO (Rank Based Overlap)

In [ ]:
pip install rbo

In [ ]:
import rbo

### Calculating RBO Rank correlation (base rank, BM25 rank)

In [ ]:
new_rank = list(BM25_final["doc_no"].astype(int))
rbo.RankingSimilarity(base_rank, new_rank).rbo()

0.3855725547401186

### Calculating RBO Rank correlation (base rank, TF-IDF rank)

In [ ]:
new_rank = list(TF_IDF_final["doc_no"].astype(int))
rbo.RankingSimilarity(base_rank, new_rank).rbo()

0.465611826639069

### Calculating RBO Rank correlation (base rank, TF-IDF with cosine similarity rank)

In [ ]:
new_rank = list(TF_IDF_cos["doc_no"].astype(int))
rbo.RankingSimilarity(base_rank, new_rank).rbo()

0.37347273167558437

### Calculating RBO Rank correlation (base rank, get_tweet_score function rank)

In [ ]:
new_rank = list(scoring_fn_final["doc_no"].astype(int))
rbo.RankingSimilarity(base_rank, new_rank).rbo()

0.8565476781057152

### Calculating RBO Rank correlation (base rank, PL2 rank)

In [ ]:
new_rank = list(pl2["docno"].astype(int))
rbo.RankingSimilarity(base_rank, new_rank).rbo()

0.5038684514248314

### Calculating RBO Rank correlation (base rank, inl2 rank)

In [ ]:
new_rank = list(inl2["docno"].astype(int))
rbo.RankingSimilarity(base_rank, new_rank).rbo()

0.4763911446585368

In [ ]:
cols = list(df_final.columns.values)
cols.insert(0,'doc_no')
cols = cols[:-1]
cols

['doc_no',
 'username',
 'description',
 'location',
 'following',
 'followers',
 'totaltweets',
 'retweetcount',
 'text',
 'url',
 'hashtags',
 'title',
 'body',
 'score',
 'bm25_score',
 'tf_idf_score',
 'cos_sim_score']

In [ ]:
df_final = df_final[cols]
df_final.head()

,doc_no,username,description,location,following,followers,totaltweets,retweetcount,text,url,hashtags,title,body,score,bm25_score,tf_idf_score,cos_sim_score
0,1,FoxNews,"Follow America's #1 cable news network, delive...",U.S.A.,263,21203405,464029,1656,Elon Musk confirms he turned down Bill Gates o...,https://www.foxbusiness.com/business-leaders/e...,[],Elon Musk confirms he turned down Bill Gates o...,Fox Business\nPersonal Finance\nEconomy\nMarke...,129504.757454,0.000000,0.057230,0.000000
1,2,WatcherGuru,Watcher Guru gives you unparalleled coverage o...,"Texas, USA",20,806907,6417,456,#Dogecoin developers sent the first $Doge tran...,https://t.co/lb7vfsNN7k,['Dogecoin'],This Dogecoin transaction happened without acc...,GENERAL CRYPTO\n2 minute read\nThis Dogecoin t...,61756.112094,0.000000,0.023043,0.000000
2,3,Valeurs,L'hebdo de la droite qui s'assume. Nos réseaux...,France,179,335706,101682,915,"🔴 “Le virus woke rend Netflix irregardable”, t...",https://www.valeursactuelles.com/societe/le-vi...,[],"“Le virus woke rend Netflix irregardable”, tac...",Se connecter\nS’abonner\nAccueil > Société > “...,3538.069466,11.720660,0.036308,0.083547
3,4,CNBC,First in business worldwide.,"Englewood Cliffs, NJ",854,4637382,493166,952,Elon Musk says he confronted Bill Gates about ...,https://www.cnbc.com/2022/04/23/elon-musk-twee...,[],Elon Musk tweets that he confronted Bill Gates...,MARKETS\nBUSINESS\nINVESTING\nTECH\nPOLITICS\n...,9255.036619,0.000000,0.057485,0.000000
4,5,RBReich,"Berkeley professor, former Secretary of Labor....","Berkeley, CA",640,1489875,24091,486,Elon Musk’s real goal has nothing to do with t...,https://www.theguardian.com/commentisfree/2022...,[],Elon Musk wants to own Twitter to protect his ...,Skip to main content\nSkip to navigation\nSupp...,4014.861106,1.265734,0.053386,0.002485
